In [28]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas as pd

# all units SI (meters, kg, etc.)
distance = 7
height = 2.495
m = 0.141748
R_l = 0.0508
R_u = 0.03175
R_b = 0.0889
A = 0.0248286665
interval = 0.001
run = True

## Create Functions

In [29]:
def iterate():
    global run
    for w_r in range(30,600): #6380 falcon 500 rpm in radians/s
        for theta in range(15,85):
            if(run):
                trajectory(w_r, theta*math.pi/180)
    #print("done")
    run = True

In [30]:
def trajectory(w, theta):
    v = (w*R_l + 9/16*w*R_u)/2
    #print(v)
    s = (0.5)*(w*R_l - 9/16*w*R_u)/(2*math.pi*R_b)
    runPath(w,v,s,theta)

In [31]:
def runPath(w,v_i,s,theta_i):
    t = 0
    v_x = v_i*math.cos(theta_i)
    v_y = v_i*math.sin(theta_i)
    theta = theta_i
    x_dist = 0
    y_dist = 0
    through = True
    x = [0]
    y = [0]
    while(through):
        x_dist += v_x*interval
        y_dist += v_y*interval
        x.append(x_dist)
        y.append(y_dist)
        v_n = math.sqrt(v_x*v_x + v_y*v_y)
        theta = angle(v_x,v_y)
     
        magnus = magnusF(s, v_n)
        drag = dragF(v_n)
        v_x -= (magnus*math.sin(theta) + drag*math.cos(theta))*interval
        v_y -= (drag*math.sin(theta) + 9.8 - magnus*math.cos(theta))*interval
        t += interval
        
        if(v_y < 0.1 and v_y > -0.1):
            if(((x_dist > distance - .01) and (x_dist < distance + .01)) 
            and ((y_dist > height - .015) and (y_dist < height + .015))):
                through = False
                global w_min
                plt.plot(x,y)
                omega_array.append(round(w*60/(2*math.pi),2))
                theta_array.append(round(theta_i*180/math.pi,2))
                velocity_array.append(round(v_i,2))
                spin_array.append(round(s,2))
                time_array.append(round(t,3))
                print ("motor_rpm: " + str(round(w*60/(2*math.pi),2)) + " angle: " + str(round(theta_i*180/math.pi,2)))
                print("velocity: " + str(round(v_i,2)) + " spin: " + str(round(s,2)) + " time: " + str(round(t,3)))
                print()
                global run
                run = False
  
        if(v_y < -0.15):
            through = False
      

In [ ]:
def magnusF(s,v):
    return (0.1)*(16/3)*(math.pi*math.pi)*1.229*(R_b**3)*s*v/m
def dragF(v):
    return (0.47)*(1.229/2)*A*(v**2)/m
def angle(x,y):
    return math.atan(y/x)

In [ ]:
plt.title("Peak-Based Ball Trajectory at Varying Distances, \n$C_l$ = 0.1, $C_f$ = 0.47")
plt.xlabel("Distance (m)")
plt.ylabel("Height (m)")

plt.rcParams.update({'mathtext.default':  'regular' })

trial_array = []
distance_array = []
omega_array = []
theta_array = []
velocity_array = []
spin_array = []
time_array = []
max_trials = 30

for i in range(0,max_trials):
    trial_array.append(i+1)
    distance = 2.3 + ((7-2.3) / max_trials)*i
    distance_array.append(distance)
    print("trial: " + str(i+1) + " , " + "distance: " + str(round(distance,3)))
    iterate()

plt.savefig("peak_based_" + str(max_trials) + "_trials.png")
plt.show()

trial: 1 , distance: 2.3
motor_rpm: 2301.38 angle: 61.0
velocity: 8.27 spin: 7.11 time: 0.707

trial: 2 , distance: 2.457
motor_rpm: 2320.48 angle: 60.0
velocity: 8.34 spin: 7.17 time: 0.722

trial: 3 , distance: 2.613
motor_rpm: 2368.23 angle: 58.0
velocity: 8.51 spin: 7.31 time: 0.71

trial: 4 , distance: 2.77
motor_rpm: 2387.32 angle: 57.0
velocity: 8.58 spin: 7.37 time: 0.724

trial: 5 , distance: 2.927
motor_rpm: 2425.52 angle: 56.0
velocity: 8.72 spin: 7.49 time: 0.736

trial: 6 , distance: 3.083
motor_rpm: 2473.27 angle: 54.0
velocity: 8.89 spin: 7.64 time: 0.72

trial: 7 , distance: 3.24
motor_rpm: 2501.92 angle: 53.0
velocity: 8.99 spin: 7.73 time: 0.732

trial: 8 , distance: 3.397
motor_rpm: 2559.21 angle: 51.0
velocity: 9.2 spin: 7.9 time: 0.719

trial: 9 , distance: 3.553
motor_rpm: 2587.86 angle: 50.0
velocity: 9.3 spin: 7.99 time: 0.726

trial: 10 , distance: 3.71
motor_rpm: 2626.06 angle: 49.0
velocity: 9.44 spin: 8.11 time: 0.734

trial: 11 , distance: 3.867


In [26]:
print(len(trial_array))
print(len(distance_array))
print(len(omega_array))
print(len(theta_array))

29
29
29
29


In [25]:
trial_array.remove(28)
distance_array.remove(6.53)

## Test

In [27]:
def make_bold(val):
    bold = 'bold'
    
    return 'font-weight: %s' % bold

df = pd.DataFrame(data = {'Trial': trial_array, 
                          'Distance': distance_array, 
                          'Angular Velocity': omega_array, 
                          'Angle': theta_array,
                          'Velocity': velocity_array,
                          'Spin': spin_array,
                          'Time': time_array})
print(df)
df.to_csv("peak_based_" + str(max_trials) + "_trials.csv")

    Trial  Distance  Angular Velocity  Angle  Velocity   Spin   Time
0       1  2.300000           2263.18   59.0      8.14  13.98  0.747
1       2  2.456667           2291.83   57.0      8.24  14.15  0.734
2       3  2.613333           2310.93   56.0      8.31  14.27  0.755
3       4  2.770000           2349.13   54.0      8.45  14.51  0.741
4       5  2.926667           2377.77   53.0      8.55  14.68  0.757
5       6  3.083333           2406.42   52.0      8.65  14.86  0.772
6       7  3.240000           2444.62   50.0      8.79  15.10  0.758
7       8  3.396667           2473.27   49.0      8.89  15.27  0.771
8       9  3.553333           2501.92   48.0      8.99  15.45  0.783
9      10  3.710000           2549.66   46.0      9.17  15.75  0.767
10     11  3.866667           2587.86   45.0      9.30  15.98  0.775
11     12  4.023333           2616.51   44.0      9.41  16.16  0.785
12     13  4.180000           2645.16   43.0      9.51  16.34  0.795
13     14  4.336667           2683